In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import sys
from PIL import Image
from tensorflow import keras

In [2]:
base_dir = os.getcwd()+"/data"
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

In [3]:
batch_size = 32
image_size = 160
# Rescale all images by 1./255 and apply image augmentation
train_datagen = keras.preprocessing.image.ImageDataGenerator(
                rescale=1./255)

validation_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)


In [4]:
# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
                train_dir,  # Source directory for the training images
                target_size=(image_size, image_size),  
                batch_size=batch_size,
                # Since we use binary_crossentropy loss, we need binary labels
                class_mode='categorical')

# Flow validation images in batches of 20 using test_datagen generator
validation_generator = validation_datagen.flow_from_directory(
                validation_dir, # Source directory for the validation images
                target_size=(image_size, image_size),
                batch_size=batch_size,
                class_mode='categorical')

Found 4800 images belonging to 6 classes.
Found 1176 images belonging to 6 classes.


In [5]:
IMG_SHAPE = (image_size, image_size, 3)

# Create the base model from the pre-trained model MobileNet V2
base_model = tf.keras.applications.VGG16(input_shape=IMG_SHAPE,
                                               include_top=False, 
                                               weights='imagenet')

In [6]:
base_model.trainable = False

In [7]:
base_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 160, 160, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 160, 160, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 160, 160, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 80, 80, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 80, 80, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 80, 80, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 40, 40, 128)       0         
__________

In [8]:
model = tf.keras.Sequential([
  base_model,
  keras.layers.GlobalAveragePooling2D(),
  keras.layers.Dense(6, activation='softmax')
])

In [9]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [10]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 5, 5, 512)         14714688  
_________________________________________________________________
global_average_pooling2d_1 ( (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 3078      
Total params: 14,717,766
Trainable params: 3,078
Non-trainable params: 14,714,688
_________________________________________________________________


In [11]:
len(model.trainable_variables)

2

In [12]:
epochs = 10
steps_per_epoch = train_generator.n // batch_size
validation_steps = validation_generator.n // batch_size

model.fit_generator(train_generator, 
                              steps_per_epoch = steps_per_epoch,
                              epochs=epochs, 
                              workers=4,
                              validation_data=validation_generator, 
                              validation_steps=validation_steps)

Epoch 1/10
150/150 [==============================] - 1515s 10s/step - loss: 1.8453 - acc: 0.1819 - val_loss: 1.7345 - val_acc: 0.2361
Epoch 2/10
150/150 [==============================] - 1577s 11s/step - loss: 1.6622 - acc: 0.3958 - val_loss: 1.6163 - val_acc: 0.5490
Epoch 3/10
150/150 [==============================] - 1574s 10s/step - loss: 1.5573 - acc: 0.7217 - val_loss: 1.5263 - val_acc: 0.7640
Epoch 4/10
150/150 [==============================] - 1392s 9s/step - loss: 1.4614 - acc: 0.8952 - val_loss: 1.4350 - val_acc: 0.8951
Epoch 5/10
150/150 [==============================] - 1366s 9s/step - loss: 1.3702 - acc: 0.9563 - val_loss: 1.3545 - val_acc: 0.9205
Epoch 6/10
150/150 [==============================] - 1366s 9s/step - loss: 1.2832 - acc: 0.9633 - val_loss: 1.2656 - val_acc: 0.9860
Epoch 7/10
150/150 [==============================] - 1365s 9s/step - loss: 1.2005 - acc: 0.9829 - val_loss: 1.1935 - val_acc: 0.9886
Epoch 8/10
150/150 [==============================] - 1368s

In [13]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

In [14]:
model.save_weights("model.h5")

In [16]:
from keras.models import model_from_json
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")

Using TensorFlow backend.
